In [1]:
! pip install darts

     |████████████████████████████████| 592 kB 944 kB/s eta 0:00:01
     |████████████████████████████████| 825 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 7.9 MB/s eta 0:00:011
     |████████████████████████████████| 195 kB 18.6 MB/s eta 0:00:01
     |████████████████████████████████| 43 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 8.9 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 601 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 22.0 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 436 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.8 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 5.8 MB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 9.6 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 

In [4]:
! brew install libomp

Running `brew update --preinstall`...
==> Downloading https://ghcr.io/v2/homebrew/portable-ruby/portable-ruby/blobs/sha256:1f50bf80583bd436c9542d4fa5ad47df0ef0f0bea22ae710c4f04c42d7560bca
######################################################################### 100.0%
==> Pouring portable-ruby-2.6.8_1.el_capitan.bottle.tar.gz
==> Auto-updated Homebrew!
Updated 1 tap (homebrew/core).
==> New Formulae
adamstark-audiofile                      macpine
age-plugin-yubikey                       mailsy
agg                                      manifest-tool
akku                                     mariadb@10.7
ansible@6                                mariadb@10.8
antidote                                 mariadb@10.9
apophenia                                markdown-toc
aptos                                    markdownlint-cli2
aribb24                                  marksman
arxiv_latex_cleaner                      mbt
astro                                    mbw
autocorrect                   

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import mape, r2_score
from darts.datasets import EnergyDataset

In [6]:
def display_forecast(pred_series, ts_transformed, forecast_type, start_date=None):
    plt.figure(figsize=(8, 5))
    if start_date:
        ts_transformed = ts_transformed.drop_before(start_date)
    ts_transformed.univariate_component(0).plot(label="actual")
    pred_series.plot(label=("historic " + forecast_type + " forecasts"))
    plt.title(
        "R2: {}".format(r2_score(ts_transformed.univariate_component(0), pred_series))
    )
    plt.legend()

In [7]:
input = pd.read_csv('../kaggle_data/input.csv')

In [8]:
input

,Unnamed: 0,1001,1003,1005,1007,1009,1011,1013,1015,1017,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
0,2019-08-01,3.007682,7.239156,1.073138,1.310777,1.544148,0.926372,0.674981,2.691544,1.486542,...,1.150575,5.916088,4.062681,30.183279,6.115724,2.640386,26.433363,4.612016,2.603246,1.548594
1,2019-09-01,2.884870,7.290936,0.995794,1.305176,1.575892,0.998556,0.739265,2.703752,1.355156,...,1.150575,5.894528,3.888567,31.712736,6.142085,2.858371,26.611856,4.338306,2.683593,1.530794
2,2019-10-01,3.055843,7.425439,1.160149,1.254761,1.546415,0.974495,0.713551,2.693764,1.355156,...,1.150575,5.881592,3.903076,31.990431,6.181626,2.913635,26.736261,4.420419,2.635385,1.530794
3,2019-11-01,2.993233,7.426071,1.000628,1.254761,1.573625,1.022618,1.054256,2.739270,1.362664,...,1.200600,5.954897,3.845038,32.776520,6.234348,3.008812,26.920164,4.372520,2.683593,1.601994
4,2019-12-01,2.993233,7.470274,1.000628,1.265965,1.555485,1.034649,1.054256,2.729281,1.343894,...,1.200600,5.911776,3.845038,32.806427,6.273890,3.088637,26.903938,4.310935,2.635385,1.584194
5,2020-01-01,2.969090,7.413655,1.027229,1.253638,1.573217,1.060717,1.060871,2.719628,1.319910,...,1.218893,5.901441,3.890260,32.847649,6.353848,3.150922,26.684565,4.367864,2.666882,1.590706
6,2020-02-01,2.909326,7.282522,1.022314,1.248041,1.536999,1.036333,1.060871,2.745211,1.331096,...,1.269680,5.871155,3.716069,32.246933,6.486774,3.169494,26.738255,4.237172,2.699208,1.519214
7,2020-03-01,2.933231,7.309610,1.032144,1.264831,1.541526,1.060717,1.009121,2.756335,1.301268,...,1.269680,5.897114,3.527362,32.856171,7.018477,3.237588,26.668457,4.182143,2.699208,1.537087
8,2020-04-01,3.000167,7.568799,1.086209,1.270428,1.577744,1.011948,0.996183,2.818625,1.327368,...,1.269680,5.979319,3.469299,34.142807,6.965306,3.293302,27.119463,4.195900,2.683045,1.554960
9,2020-05-01,3.004948,7.603275,1.081294,1.287217,1.575481,0.890027,0.996183,2.834197,1.342282,...,1.320467,6.005279,3.483815,40.916836,7.257743,3.311873,27.414764,4.443527,2.731534,1.608579


In [9]:
x = pd.DataFrame(input.iloc[:, 0:2])

In [10]:
x

,Unnamed: 0,1001
0,2019-08-01,3.007682
1,2019-09-01,2.884870
2,2019-10-01,3.055843
3,2019-11-01,2.993233
4,2019-12-01,2.993233
5,2020-01-01,2.969090
6,2020-02-01,2.909326
7,2020-03-01,2.933231
8,2020-04-01,3.000167
9,2020-05-01,3.004948


In [11]:
input = TimeSeries.from_dataframe(x, time_col='Unnamed: 0')

In [12]:
model_nbeats = NBEATSModel(
    input_chunk_length=30,
    output_chunk_length=7,
    generic_architecture=True,
    num_stacks=10,
    num_blocks=1,
    num_layers=4,
    layer_widths=512,
    n_epochs=100,
    nr_epochs_val_period=1,
    batch_size=800,
    model_name="nbeats_run",
)

In [13]:
model_nbeats.fit(input, verbose=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 8.1 M 
---------------------------------------------------
8.1 M     Trainable params
2.7 K     Non-trainable params
8.1 M     Total params
64.735    Total estimated model params size (MB)


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  9.75it/s, loss=0.000661, train_loss=0.000637]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  9.53it/s, loss=0.000661, train_loss=0.000637]
